In [11]:
from llama_index.core import SimpleDirectoryReader

from llama_index.readers.file.markdown import MarkdownReader

parser = MarkdownReader()
file_extractor = {".md": parser}
documents = SimpleDirectoryReader(
    input_files= ["/home/drunkwcodes/projects/myfoam/kb/package_manager/pdm.md"], file_extractor=file_extractor
).load_data()
# documents = SimpleDirectoryReader(input_files= ["/home/drunkwcodes/projects/myfoam/kb/package_manager/pdm.md"]).load_data()

In [38]:
from llama_index.readers.file import PDFReader

# PDF Reader with `SimpleDirectoryReader`
parser = PDFReader()
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(
    input_files= ["/home/drunkwcodes/Documents/大家的日本语1-2册教材/01.pdf"], file_extractor=file_extractor
).load_data()

In [12]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding()

In [13]:
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

In [39]:
# create client
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("japanese")

In [40]:
# save embedding to disk
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [41]:
# create index
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

# The VectorStoreIndex class, in conjunction with the storage_context (which points to the ChromaDB collection), 
# stores these embeddings and their corresponding document metadata in the specified ChromaDB collection.

In [42]:
# load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("japanese")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)

In [20]:

from llama_index.llms.ollama import Ollama

In [46]:
llm = Ollama(model="llama3.2", request_timeout=120)

In [47]:
# Query Data from the persisted index
query_engine = index.as_query_engine(llm=llm)

In [48]:
response = query_engine.query("工程師的日文？")
print(response)

工程師的日文包括以下几个方面：

1.  **職業關係語**：例如「先生」、「主任」、「工程師」，使用於與對方的職業或地位建立關係時。
2.  **工作相關詞彙**：如“設計”、“建造”、“檢查”等，描述工程师的工作過程和實際。
3.  **日常生活中可能遇到的情況**：例如“休息”、“旅行”、“購物”，使用於描述工程师在工作之外的生活經歷。
4.  **專業關係語**：如“報告”、“評估”、“監管”等，描述工程师與對方之間的職業關係和互動。

以下是一些日文例句：

*   「こんにちは、 Mr. Smith-san です。私はエンジニアです。」（中文翻譯：「hello，Mr. Smith先生，我是工程師。"])
*   「このプロジェクトでは、設計と建造を担当することになります。」（中文翻譯：`這個項目中，我們負責設計和建造。`)
*   「今日は休息日です。 tomorrow は仕事に戻ります。」（中文翻譯：`今天是休息日，明天我們再回工作。`)
*   「このプロジェクトの評価をします」という言葉で「報告」、「評估」的意義被表達出。
*   「彼はエンジニアです。彼が仕事をしている間には、毎日同じルートを走ります。」（中文翻譯：`他人是工程師，他的工作時間每天走一樣路。`)
